In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableLambda
from bs4 import BeautifulSoup
import re

In [2]:
model_name = 'dunzhang/stella_en_400M_v5'
model_kwargs = {'device': 'cuda', "trust_remote_code": True}

embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

/home/exouser/miniconda3/envs/ai-agent-askus/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
A new version of the following files was downloaded from https://huggingface.co/dunzhang/stella_en_400M_v5:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/dunzhang/stella_en_400M_v5:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

In [3]:
vector_store = Chroma(
    collection_name="faq",
    persist_directory="./db",
    embedding_function=embedding_model,
    collection_metadata={"hnsw:space": "cosine"}
)

In [29]:
vector_store.reset_collection()

In [30]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=8000,
    chunk_overlap=100
)

In [32]:
from loaders.HTMLDirectory import HTMLDirectoryLoader
from html2text import HTML2Text

h = HTML2Text()
h.ignore_images = True

def faq_html_parser(html):
    soup = BeautifulSoup(html)
    question = soup.find(id="kb_article_question")
    answer = soup.find(id="kb_article_text")

    if not question or not answer:
        return None
    
    question = h.handle(str(question))
    answer = h.handle(str(answer))

    qa = f"{question}\n{answer}"
    removed_repeating_newlines = re.sub(r'\n{2,}', '\n', qa)

    return removed_repeating_newlines

faq_html_loader = HTMLDirectoryLoader("../web-scraper/faq-archive", faq_html_parser)
faq_documents = list(faq_html_loader.lazy_load())
faq_split_documents = text_splitter.split_documents(faq_documents)
vector_store.add_documents(faq_split_documents)

['7bee3f58-8aba-49a7-9862-094e1efe518a',
 '66e2a5bc-4bf6-4d56-b99e-9578771fdfef',
 'aa6b78d4-acfa-474d-b48f-7065345f8bfd',
 '39e75627-2bf9-4734-8ebb-6889e7a788bc',
 'd1fb96fc-10ab-4ced-8e5b-3c93833665f0',
 'f3f5c674-63b8-4199-9b6c-1c6f9ab040a0',
 '61bcd741-bdcc-4f14-b962-c55b69dfe215',
 '620fa222-290d-49c7-84a8-53acfbc7e597',
 '79082050-12db-4b0c-8ea3-3b48eed0ff4e',
 'fed13c9c-1a60-4154-a310-30900e39039f',
 '6c6b3813-7be7-4a9d-9fcb-4c2ad6ef9302',
 '9809daca-9d19-4ec1-8527-4c6fd035cb9e',
 '43d28b79-a474-4920-ada3-b9a2e1d102f9',
 '57178352-2bbe-4f89-b924-737118f93c26',
 '9ebf2cd7-83f8-48f4-a3d4-5652ba9f0a2b',
 '6ddfb987-f62e-4d30-ad05-8dc043349912',
 '9d5eb902-db7f-489b-8b6f-3b5f0b1ac9a6',
 'eedb2762-f91e-454f-b052-c07ca1715776',
 '5eceaa31-8db1-4454-80c3-f5613367acae',
 'e6ca7267-9aaf-468d-bb82-d06c13abaf21',
 'd13b9bf5-d244-4d2c-9119-a47363e38f71',
 '7aff6968-dc4c-4130-9be1-37af649d5d3a',
 '496ed44d-bfb8-4596-a2b9-20ed7258ad12',
 '167b84d5-2454-439c-add3-a1df77fd65b6',
 'ee362806-564f-

In [33]:
vector_store.similarity_search("duo mobile")

[Document(metadata={'source': 'https://www.hawaii.edu/askus/1859'}, page_content='##  Duo Mobile app 4.0 update\n  *[UH]: University of Hawaii\n  *[sFTP]: secure FTP\n  *[ITS]: Information Technology Services\n  *[FAQ]: Frequently Asked Questions\n### Overview\nThe Duo Mobile 4.0 version of the app is a significant update to the user\ninterface. This update will be released on the following schedule:\n  * For iOS: October 11 through 18, 2021\n  * For Android: October 11 through 15, 2021\nDuo Mobile 4.0 requires\n  * iOS 13 and up\n  * Android 8 and up\nOlder versions of the Duo Mobile app will continue to work.\n### Summary of changes\nThe following are some of the changes to the Duo Mobile app in version 4.0.\n**New Duo Approve screen**\nIn versions prior to 4.0, the _Approve_ button was on the left and the _Deny_\nbutton was on the right. In Duo Mobile 4.0, the _Approve_ button has been\nmoved to the **right** , and the _Deny_ button moved to the **left**. Duo made\nthis change to fo

In [ ]:
from loaders.JSONFile import JSONFileLoader

def json_parser(d):
    return {
        "page_content": d["extracted"],
        "metadata" : {"source": d["url"]}
    }

json_file_loader = JSONFileLoader("../web-scraper/data/urls.json", json_parser)
json_documents = json_file_loader.lazy_load()
json_split_documents = text_splitter.split_documents(json_documents)
vector_store.add_documents(json_split_documents)

In [4]:
llm = ChatOllama(model="llama3.1:8b-instruct-q8_0", temperature=0)

In [5]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

from langchain.globals import set_verbose
from langchain.callbacks.tracers import ConsoleCallbackHandler
set_verbose(True)


retriever = vector_store.as_retriever(
    search_kwargs={'k': 2}
)

combined_retriever = EnsembleRetriever(retrievers=[retriever, ])

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=combined_retriever
)

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question. "
    "just reformulate it if needed and otherwise return it as is. "
    "if there is no chat history, return the input as is. "
    "if the input is a greeting, return the input as is. "
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, combined_retriever, contextualize_q_prompt
)

system_prompt = (
    "Your name is Hoku. You are an assistant for answering questions about UH Manoa."
    "Answer the question given ONLY the provided context.\n"
    "If the answer DOES NOT appear in the context, say 'I'm sorry I don't know the answer to that'.\n"
    "Keep your answer concise, informative and with a conversational tone.\n"
    "DO NOT mention the context or the text provided."
    "if the user greets you, greet them back nicely"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "context:{context}\n\nquestion: {input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

sources_examples = [
    {"input": "Hi Hoku!", "output": "no"},
    {"input": "How are you?", "output": "no"},
    {"input": "What is duo mobile used for?", "output": "yes"},
    {"input": "what specs should i have for a mac laptop?", "output": "yes"},
    {"input": "Thank you!", "output": "no"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=sources_examples,
    input_variables=["input"]
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Your job is to classify a user input as needing sources 'yes' or not needing sources 'no'."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

def requires_source(inp: dict):
    chain = final_prompt | llm
    return "yes" in chain.invoke(inp).content.lower()


def add_sources_to_response_if_needed(inp: dict) -> dict:
    if not requires_source({"input" : inp["input"]}):
        return inp
    
    sources_text = "\n".join(list(set(doc.metadata["source"] for doc in inp['context'])))
    inp["answer"] = f"{inp['answer'].strip()}\n\nFor more information, check out these links\n{sources_text}"
    return inp

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

conversational_rag_chain_with_sources = conversational_rag_chain | add_sources_to_response_if_needed

In [6]:
store = {}

In [12]:
user_input = input()
if user_input:
    print(user_input)

    answer = conversational_rag_chain_with_sources.invoke(
        {"input": user_input},
        config={
            "configurable": {"session_id": "1"},
            # 'callbacks': [ConsoleCallbackHandler()]
        },
    )

    print(answer["answer"])
    print()

can you summarize our chat?
We've been discussing Google Chat and Meet at UH Manoa. I provided information on how to use these tools, including how to start conversations and video conferences. We also talked about requesting a new phone installation at your office on the Manoa campus. Let me know if you have any other questions!

For more information, check out these links
https://www.hawaii.edu/askus/200
https://www.hawaii.edu/askus/1794

